<a href="https://colab.research.google.com/github/clzndyal1/git_test/blob/master/17_Deep_Learning_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical  #one-hot encoding과 같은 결과 반환
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np, array
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [93]:
#텍스트의 토큰화

text = '해보지 않으면 해낼 수 없다'

result = text_to_word_sequence(text)
print('\n원문:\n', text)
print('\n토큰화\n', result)


원문:
 해보지 않으면 해낼 수 없다

토큰화
 ['해보지', '않으면', '해낼', '수', '없다']


In [94]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.']

In [95]:
token = Tokenizer()   #토큰화 인스턴스 생성
token.fit_on_texts(docs)

print('\n단어 카운트 :\n', token.word_counts )   #토큰한 단어의 빈도수를 계산, 튜플 리스트로 반환
print('\n문장 카운트 :\n', token.word_counts['텍스트의'], '회나옴')
print('\n문장 카운트 :\n', token.document_count)   #텍스트의 단어가 몇번 나오는지 출력
print('\n각 단어가 몇 개의 문장에 포함되어 있는가 :\n', token.word_docs)   #단어가 몇개의 문장에 나오는지
print('\n각 단어에 매겨진 인덱스 값 :\n', token.word_index)   #단어에 새겨진 인덱스값
print(token.texts_to_sequences(docs))   #각 단어의 인덱스만 추출


단어 카운트 :
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트 :
 2 회나옴

문장 카운트 :
 3

각 단어가 몇 개의 문장에 포함되어 있는가 :
 defaultdict(<class 'int'>, {'먼저': 1, '각': 1, '나누어': 1, '텍스트의': 2, '단어를': 1, '토큰화합니다': 1, '단어로': 1, '딥러닝에서': 2, '인식됩니다': 1, '토큰화해야': 1, '있습니다': 1, '사용할': 1, '수': 1, '결과는': 1, '토큰화한': 1})

각 단어에 매겨진 인덱스 값 :
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}
[[3, 1, 4, 5, 6, 7], [1, 8, 9, 2, 10], [11, 12, 2, 13, 14, 15]]


In [96]:
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'

token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [97]:
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [98]:
word_size = len(token.word_index) +1
x = to_categorical(x, num_classes=word_size)

print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


In [99]:
#영화 리뷰가 긍정인지? 부정인지 예측

model = Sequential()
model.add(Embedding(16,4))

In [100]:
docs = ['너무 재밋네요', '최고에요', '참 잘 만든 영화예요', '추천하고 싶은 영화입니다.', '한 번 더 보고싶네요',
        '글쎄요', '별로에요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요']

classes = np.array([1,1,1,1,1,0,0,0,0,0])

#1.토큰화, 데이터 전처리 과정
token = Tokenizer()
token.fit_on_texts(docs)   #리뷰 단어 토큰화
print('단어의 인덱스 :',token.word_index, len(token.word_index))   #총 단어수

#토큰한 결과를 가지고 인덱스로 문장을 변환
x = token.texts_to_sequences(docs)   #문장의 단어를 token한 결과단어의 인덱스로 변환해준다.
print(x)

#각 문장의 단어의 갯수가 다름 -> 문장 단어의 갯수를 동일하게 적용
padd_x = pad_sequences(x, 4)   #x를 각 문장의 단어 수가 4개로 통일 : 단어 수가 작으면 앞에 0 추가, 크면 자름
print(padd_x)

단어의 인덱스 : {'너무': 1, '재밋네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한': 11, '번': 12, '더': 13, '보고싶네요': 14, '글쎄요': 15, '별로에요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21} 21
[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13, 14], [15], [16], [17, 18], [19, 20], [21]]
[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [11 12 13 14]
 [ 0  0  0 15]
 [ 0  0  0 16]
 [ 0  0 17 18]
 [ 0  0 19 20]
 [ 0  0  0 21]]


In [101]:
#딥러닝 모델 생성

model = Sequential()
model.add(Embedding(len(token.word_index)+1, 8, input_length=4))   #(총단어수, 임베딩사이즈, 입력 단어수)
model.add(Flatten())   #다음 레이어를 위해 1차원 배열로 변환
model.add(Dense(1, activation='sigmoid'))   #긍정 또는 부정이므로 활성함수는 시그모이드
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 4, 8)              176       
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [102]:
#모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#모델 실행
model.fit(padd_x, classes, epochs=20)
print('Accuracy = ', model.evaluate(padd_x, classes)[1])

Epoch 1/20
1/1 [==============================] - 1s 673ms/step - loss: 0.7084 - accuracy: 0.2000
Epoch 2/20
1/1 [==============================] - 0s 10ms/step - loss: 0.7060 - accuracy: 0.3000
Epoch 3/20
1/1 [==============================] - 0s 20ms/step - loss: 0.7036 - accuracy: 0.4000
Epoch 4/20
1/1 [==============================] - 0s 19ms/step - loss: 0.7012 - accuracy: 0.4000
Epoch 5/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6988 - accuracy: 0.4000
Epoch 6/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6965 - accuracy: 0.4000
Epoch 7/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6941 - accuracy: 0.4000
Epoch 8/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6918 - accuracy: 0.5000
Epoch 9/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6895 - accuracy: 0.5000
Epoch 10/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6871 - accuracy: 0.7000
Epoch 11/20
1/1 [===========